In [6]:
import numpy as np
import scipy as sp
CapRate       = 5.4 #[%]
PurchasePrice = 2600000
LTV           = 75
intestRate    = 4 #[%]

In [44]:
pmt = -1 * np.pmt(intestRate/12./100.,30*12,PurchasePrice * LTV/100.)
print("DSCR:{0}",(PurchasePrice * CapRate/100.) / (pmt * 12))

DSCR:{0} 1.2567674427251707


<ipython-input-44-68ccf77048d8>:1: DeprecationWarning: numpy.pmt is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pmt instead (https://pypi.org/project/numpy-financial/).
  pmt = -1 * np.pmt(intestRate/12./100.,30*12,PurchasePrice * LTV/100.)


1950000.0